# Lectura de los datos.

In [1]:
import pandas as pd

Bussiness

In [2]:
df_business = pd.read_parquet('C:\\Users\\User\\OneDrive\\Escritorio\\Proyecto Final\\Feastly\\bussiness.parquet')

df_business.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44594 entries, 3 to 150339
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   44594 non-null  object 
 1   name          44594 non-null  object 
 2   state         44594 non-null  object 
 3   latitude      44594 non-null  float64
 4   longitude     44594 non-null  float64
 5   stars         44594 non-null  float64
 6   review_count  44594 non-null  int64  
 7   attributes    43888 non-null  object 
 8   categories    44594 non-null  object 
dtypes: float64(3), int64(1), object(5)
memory usage: 3.4+ MB


is_open eliminarlo

In [8]:
df_business = df_business.drop(columns=['is_open'])

In [ ]:
df_business

Checkin

In [3]:
df_checkin = pd.read_parquet('C:\\Users\\User\\OneDrive\\Escritorio\\Proyecto Final\\Feastly\\checkin.parquet')

df_checkin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131930 entries, 0 to 131929
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  131930 non-null  object
 1   date         131930 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


Metadatos

In [4]:
df_metadatos = pd.read_parquet('C:\\Users\\User\\OneDrive\\Escritorio\\Proyecto Final\\Feastly\\metadatos2.parquet')

df_metadatos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48180 entries, 68 to 3024976
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            48180 non-null  object 
 1   address         48180 non-null  object 
 2   gmap_id         48180 non-null  object 
 3   description     16189 non-null  object 
 4   latitude        48180 non-null  float64
 5   longitude       48180 non-null  float64
 6   category        48180 non-null  object 
 7   avg_rating      48180 non-null  float64
 8   num_of_reviews  48180 non-null  int64  
 9   price           24218 non-null  object 
 10  MISC            47888 non-null  object 
 11  state           48180 non-null  object 
dtypes: float64(3), int64(1), object(8)
memory usage: 4.8+ MB


In [6]:
df_tip = pd.read_parquet('C:\\Users\\User\\OneDrive\\Escritorio\\Proyecto Final\\Feastly\\tip.parquet')

df_tip.info()

<class 'pandas.core.frame.DataFrame'>
Index: 908848 entries, 0 to 908914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   user_id           908848 non-null  object        
 1   business_id       908848 non-null  object        
 2   text              908848 non-null  object        
 3   date              908848 non-null  datetime64[ns]
 4   compliment_count  908848 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 41.6+ MB


# Prueba de DAGS

Voy a probar primero a través de la notebook si funciona perfectamente los DAGS.

In [7]:
pip install apache-airflow


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/44.7 kB ? eta -:--:--
     ---------------------------------------- 44.7/44.7 kB 1.1 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached setuptools-75.3.0-py3-none-any.whl.metadata (6.9 kB)
     ---------------------------------------- 0.0/66.9 kB ? eta -:--:--
     ---------------------------------------- 6

In [11]:
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime,timedelta
import pandas as pd


def imprimir():
    print('Hola mundo')
    
#Configuración del DAG (flujo de trabajo)

default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=1),
}

with DAG(
    'ejemplo_jupyter_dag',
    default_args=default_args,
    description='Ejemplo de DAG en Jupyter',
    schedule_interval=timedelta(days=1),
    start_date=datetime(2023,1,1),
    #Desactivamos catchup para que no se ponga al día
    catchup=False
) as dag:
    
    hello_task = PythonOperator(
        task_id='hello_task',
        python_callable=imprimir,
    )
    
    hello_task.execute(context={})

C:\Users\User\AppData\Local\Temp\ipykernel_10492\2434503860.py:19: RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead. 
  with DAG(


[2024-11-01T17:52:04.399-0300] {baseoperator.py:405} WARNING - PythonOperator.execute cannot be called outside TaskInstance!
Hola mundo
[2024-11-01T17:52:04.422-0300] {python.py:240} INFO - Done. Returned value was: None


In [15]:
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime,timedelta
import pandas as pd

#Funciones para la transformación

def process_metadatos():
    df=pd.read_parquet('C:\\Users\\User\\OneDrive\\Escritorio\\Proyecto Final\\Feastly\\metadatos2.parquet')
    #Aqui podemos modificar como queremos que realice el ETL
    print(f"Procesando metadatos2, número de filas: {len(df)}")
    return df

def process_bussiness():
    df=pd.read_parquet('C:\\Users\\User\\OneDrive\\Escritorio\\Proyecto Final\\Feastly\\bussiness.parquet')
    print(f"Procesando bussiness, número de filas: {len(df)}")
    return df

def process_checkin():
    
    df=pd.read_parquet('C:\\Users\\User\\OneDrive\\Escritorio\\Proyecto Final\\Feastly\\checkin.parquet')
    print(f"Procesando checkin, número de filas: {len(df)}")
    return df

def process_tip():
    df=pd.read_parquet('C:\\Users\\User\\OneDrive\\Escritorio\\Proyecto Final\\Feastly\\tip.parquet')
    print(f"Procesando tip, número de filas: {len(df)}")
    return df

#Definimos el DAG
default_args={
    'owner':'airflow',
    'start_date':datetime(2023,1,1)
}

with DAG('parquet_processing_dag',default_args=default_args,schedule='@daily',catchup=False) as dag:
    task_metadatos2 = PythonOperator(
        task_id='process_metadatos',
        python_callable=process_metadatos
    )
    
    task_business = PythonOperator(
        task_id='process_business',
        python_callable=process_bussiness
    )

    task_tip = PythonOperator(
        task_id='process_tip',
        python_callable=process_tip
    )

    task_checkin = PythonOperator(
        task_id='process_checkin',
        python_callable=process_checkin
    )
    
    #Flujo de tareas
    
    task_metadatos2 >> task_business >> task_tip >> task_checkin